# Naomi Levine's Final Project (COM674 Summer 2020)

Have you ever been watcing a really good TV show and were drawn into a particular episode because of how good or how bad it was and wondered why that episode was more compelling (positively or negatively) than the others? Well, in this project I wanted to look at how to go about setting up an investigation of that question using computational methods. In this part I've elected to use web scraping to pull the viewer rating data from an older fan run website, organize the data in a manner that is usable, and understand what the fans of the show thought were the best and worts episodes of the series. My hope is that with that information then I will be able to build another collection of code which will be able to perform text analysis on the episodes identified in order to understand which themes stand out in those episodes as compare with the rest of the series. Beyond just a look at this one show, my larger research agenda is that this will add to my understanding of how people respond to specific utopian and community building elements within science fiction media so as to answer the question: how can we build toward utopian living by imagining the steps of community organizing for a human habitat on the Moon?

## Exploring the Ratings of "Babylon 5"

For this project I wanted to look at the fan run website "The Lurker's Guide to Babylon 5" (http://www.midwinter.com/lurk/countries/us/eplist.html) which provides detailed information about the 1990s science fiction television show "Babylon 5". As might be expected, this website is no longer being updated due to the show having ended its run in 1998 (this claim excludes the one season spin-off seriese "Crusade" which is cannon to the show, but not relevant for this analysis as it does not include the specific data of interest). My interest from this stems from two places: first, the fact that I have been watching through the series for the first time with my partner and we were curious about the data; and second, my interests in science fiction and community organizing as components of my research.

Looking at the top of any given episode page we can see something called a "P5 Rating" with a number next to it. This value is the average of all the ratings (from 0-10) given by individuals who watched the show and interacted with the website. If we were to click on the hyperlinked text "P5 Rating" (http://www.midwinter.com/lurk/p5/intro.html) we would see the instructions for how the "Poll 5" voting system was operated during the period when this website was activly updated. If we were to click on the hyperlinked text of the P5 score then we would see the raw data for the ratings for every episode. This data provides the name of the person who submitted the rating, the cumulative average rating that the individual has provided thus far, the standard deviation of their score from the average, the total number of ratings that that individual had provided up to that point, and their rating for that specific episode. These text pages were the main focus of my project.

## Acquiring the Data
The first step was to download the P5 Rating text files for every episode. 

In [2]:
import requests  
for key in range (111):
    p5_data = requests.get(f'http://www.midwinter.com/lurk/p5/{key:03d}')
    with open(f'p5_data/{key:03d}.txt', 'wb') as f:
        f.write(p5_data.content)

I chose to download all the files rather than just rely on webscraping to call the informaiton I wanted from the website each time I wanted to prerform a different anlysis because I wanted to reduce the amount of time for the kernel to run as well as make sure that if I wanted to work with this data in the future that I would not be reliant on the website still existing. 

It took a few sample runs to get this code to where it is as I had to figure out how to get from just downloading from one specific url to being able to pull form all of them. This was made easy-ish because the url for each text page was the same with only the three numbers for the episode code at the end of the url being different and those number were sequentially tied to the episode order. I was able to get the general formatting for this figured out, but I did get some help from Alden Bradford (my partner) with simplifying my code and getting the {key:03d} figured out.

This cell only needs to be run once per machine. Since I have already run this to get all the files saved into a folder within my final project folder then I will not need to run it again. It did take a minute or two for the code to download all of the files. 

## Accessing Downloaded Files

Next I needed to tell the program to access the files that I downloaded. This part was a bit smoother for me to build because I had the previous cell to work from. This cell takes all the data from the files and identifies where the names and associated data begins for each text file. Alden and I worked out that there seemed to be a consisten pattern with the headers for each file in how it used dashes to separate sections. With this knowledge I was able to go in and tell it to look for the line where those dashes show up and start reading the information from there to exclude the header information. This was a helpful finding because each text file had the names and associated data for the tables beginning on different lines so it was easier to just tell it a consistent pattern to read for in order to locate the data. Then, all of that information was stored in "the_one_list" (yes, I am mixing Lord of the Rings with Babylon 5). 

In [1]:
the_one_list = []
for key in range (111):
    with open(f'p5_data/{key:03d}.txt', 'r') as f:
        linelist = list(f)
        list_of_dashes =[]
        for index, line in enumerate(linelist):
            if line[1:3] == '--':
                list_of_dashes.append (index) 
        noheaderlist = linelist[(list_of_dashes [1])+1:] 
        the_one_list.append (noheaderlist)

## Alden's Very Helpful Contribution

So, I was not able to figure out how to parce "the_one_list" (which is a list of lists) on my own. After two days of fighting with it Alden offered his coding experience to make this part of the code for me. This step proved to be more challenging than we expected based on our initial look through the files for a few reasons. The first was that, while most files were fairly similar, there were some episodes with irregularities that were difficult to finangle due to there being unexpected entries. The second challenge was in attempting to get the list to talk to one another. 

Alden was able to get the data the data to make a fairly uniform chart displaying the names of every person who rated an episode as well as their rating under the given episode column and this was really huge in terms of getting the data to be usable. While we were not able to work out how to correct for all the irregularies, it was decided that they were not likely a siginificant impact on the types of questins I wanted to be able to ask of the data.

In [2]:
from aldens_awesome_episode_parser import parse_episodes
all_scores = parse_episodes(the_one_list)

His files are also included in this folder.

>update was made on 6/11/2020 at 11:39pm which corrects for the initial issues described above and in my presentation. 

## Identifying the Fans

The first thing I wanted to know was how many episodes had each individual (indexed by name) contributed across all the episodes and this step was easily done using pandas thanks to the json work that Alden provided. 

The code below reflects the more specific information that I wanted from the count above which was to know who provided ratings for 80 or more episodes. With 111 episodes in the series, having provided ratings for 80 or more episodes meant that an individual had provided their insight on more than 70% of the show. With no particular basis in other research, this seemed like a good cut off for me to consider those individuals "fans" (aka avid consumers of) the show. I chose to look specifically at fans of the show to avoid some bias in my later analysis from the one-off raters who either really liked or really did not like a single (or hand-full) of episodes (aka the "yelp" complainers). The code below is easily modified to adjust the parameters of who gets to be counted as a "fan" of the show in the event that I do work with this data more and look up more specific parameters for defining fans of media quantitatively, but for now I am content with the parameters I have set. 

In [3]:
import pandas as pd

num_reviews = pd.notna(all_scores).sum(axis=1).sort_values(ascending = False)
fans = num_reviews[num_reviews > 79]
print (fans)

Mike Hopkins         109
Steven Grimm         100
Casey Miller          98
Terrance L. Lewis     94
Kathy Sullivan        91
John Abbott           91
Dan Kimmel            86
Tizby Hunt-Ward       84
Allan Jenoff          82
Steve Fenster         82
Mark D. Grover        81
Mike Heffner          80
dtype: int64


## Average Perception of Babylon 5 Among Fans

Once I had the fans of the show loacted it should have been an easy process to find the mean of all of their ratings with pandas, but there was a problem that arose due to the initial text files we fed into json not being clean. If we look at episode text file (p5_data/008.txt) we see that next to the first three rows of the reviewers there is a note that the original data manager added in which specifically impacted fan Bill Sherman's data and resulting in his rating being the string "vote" rather than is actual rating of "7.00". After many hours of trouble shooting and looking up information about NaN and pandas (we thought this might have been the issuse despite mean knowing to skip those values) we found the issue I described. The fix we employed was to specify that we wanted float values. This fix works for the questions being asked of this code as it is, but if we wanted this to be fixed it would need to be fixed back in Alden's json code and that was going to take more time than was necessary for the goal of this project so I elected to move on.

In [4]:
fans_df = all_scores.loc[fans.index]
ap = fans_df.mean( axis = 1)
print(ap)

Mike Hopkins         8.540275
Steven Grimm         7.497500
Casey Miller         8.552041
Terrance L. Lewis    7.895745
Kathy Sullivan       9.460440
John Abbott          7.925275
Dan Kimmel           8.563953
Tizby Hunt-Ward      8.692262
Allan Jenoff         7.567073
Steve Fenster        8.080488
Mark D. Grover       6.141975
Mike Heffner         8.079375
dtype: float64


Overall, It seems that these 12 fans had above median perceptions of the show (on a 0-10 scale) which is largely positive. This data could be presented in a more tidy manner, but I'm leaving it here for now due to time constraints. 

# Relative Perspectives

## Which episodes did they like the least?
More specific than this, I wanted to know by how much they disliked a given episode as compared with their overall perception of the show. This data was of interest to me because this tells me not only which episodes a fan of Babylon 5 did not like, but also if their disliking of that episode could be considered significant. A significant difference between a fan rating on a particular episode and their overall perception of the show is a flag to me that that episode is worth futher thematic investigation to see what elements of that episode are unique as compared with the series. 

In [99]:
lowest_rated = fans_df.min(axis = 1)
print (lowest_rated)
low_index = fans_df.idxmin(axis = 1)
print (low_index)
dislike_score = lowest_rated - ap
dislike_score

Mike Hopkins         6.0
Steven Grimm         3.0
Casey Miller         6.0
Terrance L. Lewis    6.0
John Abbott          5.0
Kathy Sullivan       7.5
Dan Kimmel           7.0
Tizby Hunt-Ward      1.0
Steve Fenster        4.0
Allan Jenoff         4.0
Mark D. Grover       3.0
Mike Heffner         3.8
dtype: float64
Mike Hopkins         51
Steven Grimm          4
Casey Miller         15
Terrance L. Lewis    74
John Abbott          84
Kathy Sullivan        2
Dan Kimmel           15
Tizby Hunt-Ward      88
Steve Fenster         2
Allan Jenoff         51
Mark D. Grover       35
Mike Heffner         94
dtype: int64


Mike Hopkins        -2.540275
Steven Grimm        -4.497500
Casey Miller        -2.552041
Terrance L. Lewis   -1.895745
John Abbott         -2.925275
Kathy Sullivan      -1.960440
Dan Kimmel          -1.563953
Tizby Hunt-Ward     -7.692262
Steve Fenster       -4.080488
Allan Jenoff        -3.567073
Mark D. Grover      -3.141975
Mike Heffner        -4.279375
dtype: float64

As an example, lets look at Mike Heffner's last liked episode. 

Here I can see that the lowest score he gave was a 3.8 and this corresponds to episode 94. His percpetion of this episode was about 4.28 lower than his average percpetion of the show which is quite the disparity. Looking back at The Lurker's Guide, I can see that this corresponds to the episode titled "Strange Relations" (http://www.midwinter.com/lurk/countries/us/guide/094.html). In a quick glance at the plot points of the episode I can guess that this could be a response to one of the three major plot lines (1. Lochley-sheridan drama, 2. The telekenetic group's plot development intersecting with Lyta's character development, or 3. The emerging of the Londo-G'Kar side plot). Having seen this episode, I would guess that it was a combination of the first two plot lines that brought Make's score down since the first plot was some kind of unnecessary drama that felt out of place and the second plot point was lulling at this point in the season.* 

## Which episodes did they like the most?
Similar to discussion of the episodes that fans like the least, I wanted to know about the episodes that they liked the most. I think that it would be interesting to understand what episodes resonated the most with fans. 

In [100]:
highest_rated = fans_df.max(axis = 1)
print (highest_rated)
high_index = fans_df.idxmax(axis = 1)
print (high_index)
like_score = highest_rated - ap
like_score

Mike Hopkins          9.99
Steven Grimm         10.00
Casey Miller         10.00
Terrance L. Lewis    10.00
John Abbott          10.00
Kathy Sullivan       10.00
Dan Kimmel           10.00
Tizby Hunt-Ward      10.00
Steve Fenster        10.00
Allan Jenoff         10.00
Mark D. Grover        9.00
Mike Heffner         10.00
dtype: float64
Mike Hopkins         110
Steven Grimm          31
Casey Miller          31
Terrance L. Lewis     72
John Abbott           60
Kathy Sullivan         5
Dan Kimmel           110
Tizby Hunt-Ward       20
Steve Fenster         54
Allan Jenoff          31
Mark D. Grover        66
Mike Heffner          22
dtype: int64


Mike Hopkins         1.449725
Steven Grimm         2.502500
Casey Miller         1.447959
Terrance L. Lewis    2.104255
John Abbott          2.074725
Kathy Sullivan       0.539560
Dan Kimmel           1.436047
Tizby Hunt-Ward      1.307738
Steve Fenster        1.919512
Allan Jenoff         2.432927
Mark D. Grover       2.858025
Mike Heffner         1.920625
dtype: float64

Interestingly, we don't see has high of discrepencies between among this data as there was with the episodes that fans liked the least. The rating that stands out the most to me is from Mark D. Grover who was more fond of episode 66 than his average perception of the series overall by about 2.86 (on a scale from 0-10). This episode, titled "Za'ha'dum", was a pinnacle plot episode for the series as it concluded the major arc of seasons 2 and 3 and set up the calm before the next major story arc of the series. My guess would be that Mark's preference for this episode is tied to just how many major plot points culminated at the end of this episode.**

*The fact that I'm guessing at what fans are responding to brings up an important point of further research consideration with this data. I would be able to get more specific information about elements of an episode that these fans could have been responding to using text analysis. This was an additional layer for this project that I had considered, but after the hefty amount of work that was put into trying to get the data usable I did not have the time to build that component. In the files for this project, you will notice the notebook titled "Episode_text_analysis" and that does have my initial outlining notes for this section so you may be interested in taking a look at some of my early ideas for this section. 

** Similar to the point made above, I would not have to guess if I were do build the text analysis component of this projec that I had made some plans for initially. However, seeing this particular data point does allow me to see that I should not just be look for episodes that have unique themes as compared to the rest of the series, but also episodes that share the most themes with the series overall because those episodes are going to be ones where arcs come to a close and, as I can now see with Mark, that is also something that fan have stronger responses to as compared to their overall perception of the show. 